In [43]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert import HTMLExporter
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatText, Button, Output, VBox, HBox, Layout

# Custom CSS for styling
style = """
<style>
.title-bar {
    background-color: #4CAF50;
    color: white;
    text-align: center;
    padding: 10px 0;
    font-size: 20px;
    font-weight: bold;
}
.title-bar1 {
   background-color: #4CAF50;
    color: white;
    text-align: center;
    padding: 5px 0;
    font-size: 10px;
    font-weight: bold;
   
}
.sidebar {
    background-color: #f2f2f2;
    width: 200px;
    padding: 10px;
    position: fixed;
    height: 100%;
    overflow: auto;
}
.button {
     background-color: #008CBA;
    color: white;
    padding: 5px 20px 15px 20px; /* More padding at the bottom */
    text-align: center;
    text-decoration: none;
    display: inline-block;
    font-size: 16px;
    margin: 4px 2px;
    cursor: pointer;
    border-radius: 8px;
    border: none;
    min-width: 350px; /* Minimum width */
}
.button:hover {
    background-color: #005f73;
}
</style>
"""

# Title and Sidebar
title_html = "<div class='title-bar'>Smart Delinquency Strategist - Predict Prescribe & Adapt</div>"
display(HTML(style + title_html ))


body_html = "<h2>Steps for Predicting and accountwise prescriptive action</h2>"
display(HTML(body_html ))


# Function to execute a notebook and return its output as HTML
def execute_notebook(notebook_path):
    with open(notebook_path) as f:
        nb = nbformat.read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=60000, kernel_name='python3')
    try:
        ep.preprocess(nb, {'metadata': {'path': './'}})
    except Exception as e:
        return f"<p>Error executing the notebook: {str(e)}</p>"

    html_exporter = HTMLExporter()
    html_exporter.exclude_input = True
    (body, resources) = html_exporter.from_notebook_node(nb)

    return body




# Create a button for each notebook and set up the event handler
notebooks = ["Synthetic_data_generation.ipynb", 
             "Data_Pre_Processing_and_Data_Analysis.ipynb", 
             "Model_Building_And_Selection.ipynb",
             "Prescriptive_modelling1.ipynb","Calculate_Man_Power.ipynb"]

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        html_content = execute_notebook(b.description)
        out_html = "<h2>Output for Steps Executed</h2>"
        display(HTML(out_html ))
        display(HTML(html_content))
        

buttons = [widgets.Button(description=nb, button_style='info') for nb in notebooks]
for button in buttons:
    button.add_class('button')  # Apply custom CSS class
    button.on_click(on_button_clicked)
    display(button)

# Output area for displaying notebook results

display(output)


Button(button_style='info', description='Synthetic_data_generation.ipynb', style=ButtonStyle(), _dom_classes=(…

Button(button_style='info', description='Data_Pre_Processing_and_Data_Analysis.ipynb', style=ButtonStyle(), _d…

Button(button_style='info', description='Model_Building_And_Selection.ipynb', style=ButtonStyle(), _dom_classe…

Button(button_style='info', description='Prescriptive_modelling1.ipynb', style=ButtonStyle(), _dom_classes=('b…

Button(button_style='info', description='Calculate_Man_Power.ipynb', style=ButtonStyle(), _dom_classes=('butto…

Output()

### Constraint responsive modelling
#### 1. After running the above 5 steps, the man power requirement for the actions mentioned in the prescriptive modelling will be displayed for each zone
#### 2. In the below boxes , the available man power for each zones should be entered
#### 3. The model will respond to the availabe man power and reassign the actions within the availabel man power


In [14]:

manpower_requirements = {
    'SMS': 0.05,  
    'Calling': 0.50,
    'Notice': 1.5,
    'Direct_Visit': 3
}

In [15]:
def revise_actions4(df,available_manpower):
    df_sorted = df.sort_values(by='Balance_T', ascending=False)
    cutoff = len(df_sorted) // 2

    # Calculate total manpower required for each zone based on best action
    total_manpower_required = df_sorted.groupby('zone')['manpower_required'].sum()

    # Initial allocation and calculations
    initial_manpower_limit = {zone: 0.6* available_manpower[zone] for zone in ['North', 'South', 'East', 'West']}
    remaining_manpower_60 = initial_manpower_limit.copy()
    df_sorted['final_action'] = None
    df_sorted['manpower_final'] = 0

    for zone in ['North', 'South', 'East', 'West']:
        # Check if total available manpower is more than total required
        if available_manpower[zone] >= total_manpower_required.get(zone, 0):
            # No action needed for this zone, set final_action to best_action
            df_sorted.loc[df_sorted['zone'] == zone, 'final_action'] = df_sorted['best_action']
            df_sorted.loc[df_sorted['zone'] == zone, 'manpower_final'] = df_sorted['manpower_required']
            continue

        # Initial allocation within 90% manpower limit
        for index, row in df_sorted[(df_sorted['zone'] == zone) & (df_sorted.index < cutoff)].iterrows():
            if remaining_manpower_60[zone] >= row['manpower_required']:
                df_sorted.at[index, 'final_action'] = row['best_action']
                df_sorted.at[index, 'manpower_final'] = row['manpower_required']
                remaining_manpower_60[zone] -= row['manpower_required']

        # Remaining 40% of manpower allocation
        remaining_manpower_40 = available_manpower[zone] - (initial_manpower_limit[zone] - remaining_manpower_60[zone])

        # Step 4: Initially assign 'SMS' to the remaining accounts
        for index, row in df_sorted[(df_sorted['zone'] == zone) & (df_sorted['final_action'].isna())].iterrows():
            if remaining_manpower_40 >= manpower_requirements['SMS']:
                df_sorted.at[index, 'final_action'] = 'SMS'
                df_sorted.at[index, 'manpower_final'] = manpower_requirements['SMS']
                remaining_manpower_40 -= manpower_requirements['SMS']

        # Step 5: Consider assigning second best action
        actions = ['SMS', 'Calling', 'Notice', 'Direct_Visit']
        for index, row in df_sorted[(df_sorted['zone'] == zone) & (df_sorted['final_action'] == 'SMS')].iterrows():
            # Find the second-best action
            action_uplifts = {action: row[f'uplift_{action}'] for action in actions if action != row['best_action']}
            second_best_action = max(action_uplifts, key=action_uplifts.get)
            if remaining_manpower_40 >= manpower_requirements[second_best_action]:
                df_sorted.at[index, 'final_action'] = second_best_action
                df_sorted.at[index, 'manpower_final'] = manpower_requirements[second_best_action]
                remaining_manpower_40 -= manpower_requirements[second_best_action]

    return df_sorted


In [40]:

    reassign_button = Button(description='Reassign Actions')
    reassign_button.add_class('button')
    output = Output()
    

    manpower_widgets = {zone: FloatText(value=0.0, description=f'{zone} Available Manpower:') for zone in ['North', 'South', 'East', 'West']}


   

    def on_reassign_button_clicked(b):
        df = pd.read_csv('final_oms_data.csv')  
        available_manpower = {zone: widget.value for zone, widget in manpower_widgets.items()}
        revised_df = revise_actions4(df,available_manpower)


        with output:
            output.clear_output(wait=True)
            display(revised_df.head())

                       
             # Calculate the total manpower required and used for each zone
            total_manpower_original = revised_df.groupby('zone')['manpower_required'].sum()
            total_manpower_final = revised_df.groupby('zone')['manpower_final'].sum()

        # Create a DataFrame for comparison
            comparison_df = pd.DataFrame({
            'Zone': total_manpower_original.index,
            'Original Manpower Required': total_manpower_original.values,
            'Available Manpower': [available_manpower[zone] for zone in total_manpower_original.index],
            'Final Manpower Used': total_manpower_final.values
            })

      # Display the comparison table
            display(comparison_df)
        
            zones = ['North', 'South', 'East', 'West']
            for zone in zones:
                plt.figure(figsize=(10, 5))

                # Get counts of final_action and best_action for the zone
                final_action_counts = revised_df[revised_df['zone'] == zone]['final_action'].value_counts()
                best_action_counts = revised_df[revised_df['zone'] == zone]['best_action'].value_counts()

            # Create a DataFrame for plotting
                action_comparison_df = pd.DataFrame({
                'Final Action': final_action_counts,
                'Best Action': best_action_counts
                }).fillna(0)

            # Plot
                action_comparison_df.plot(kind='bar', color=['lightgreen', 'lightblue'])
                plt.title(f'Distribution of Actions in {zone} Zone')
                plt.xlabel('Actions')
                plt.ylabel('Count')
                plt.show()
            

    reassign_button.on_click(on_reassign_button_clicked)


    
    widgets_layout = VBox([HBox([manpower_widgets[zone]]) for zone in ['North', 'South', 'East', 'West']] + [reassign_button])
    display(VBox([widgets_layout, output]))
    
    